In [1]:
import random

# List of 100 different stars (you can extend this list as needed)
stars = [
    "Sirius", "Canopus", "Rigil Kentaurus", "Arcturus", "Vega", "Capella", "Rigel",
    "Procyon", "Achernar", "Betelgeuse", "Hadar", "Altair", "Aldebaran", "Spica", 
    "Antares", "Pollux", "Fomalhaut", "Deneb", "Mimosa", "Regulus", "Adhara", 
    "Shaula", "Castor", "Gacrux", "Bellatrix", "Alnilam", "Alnair", "Alnitak", 
    "Eta Carinae", "Alphard", "Algol", "Dubhe", "Wezen", "Sargas", "Alhena", 
    "Kaus Australis", "Peacock", "Atria", "Markab", "Alcor", "Menkalinan", "Mirfak",
    "Gienah", "Acrux", "Alcyone", "Pherkad", "Maia", "Menkent", "Merak", "Dschubba", 
    "Zeta Reticuli", "Izar", "Zubenelgenubi", "Denebola", "Mintaka", "Caph", "Arctans",
    "Enif", "Ruchbah", "Shedar", "Scheat", "Sualocin", "Tarazed", "Ankaa", "Nunki",
    "Mirzam", "Saiph", "Hamal", "Acamar", "Phecda", "Baten Kaitos", "Almach", "Kochab",
    "Navi", "Unukalhai", "Sabik", "Tiaki", "Furud", "Aldhibah", "Mira", "Castula",
    "Tureis", "Maia", "Merope", "Sterope", "Atlas", "Pleione", "Electra", "Celaeno", 
    "Taygeta", "Alpheratz", "Rasalhague", "Rastaban", "Muphrid", "Kornephoros", "Sadalmelik" , "Venus" , "Mars","Jupiter" ,  "Saturn"
 , "Uranus" , "Neptune" ,"Pluto"]

# List of different sentence templates where the star names can be inserted
sentence_templates = [
    "Point the telescope at {}.",
    "Show me {} in the sky.",
    "Where is {} located?",
    "Find the position of {}.",
    "What are the coordinates of {}?",
    "{} is one of the brightest stars.",
    "Can you locate {}?",
    "Is {} visible tonight?",
    "We need to find {}.",
    "The star {} is quite famous.",
    "Astronomers often observe {}."
]

# Function to generate training data for 100 stars
def generate_training_data(star_list, template_list):
    train_data = []
    for star in star_list:
        # Randomly select a sentence template
        sentence_template = random.choice(template_list)
        # Insert the star name into the sentence
        sentence = sentence_template.format(star)
        # Find the start and end position of the star name in the sentence
        start_idx = sentence.index(star)
        end_idx = start_idx + len(star)
        # Create a training data tuple with the sentence and the entity annotations
        train_data.append((sentence, {"entities": [(start_idx, end_idx, "CELESTIAL_OBJECT")]}))
    return train_data

# Generate the training data
train_data = generate_training_data(stars, sentence_templates)

# Example of generated training data
for entry in train_data[:5]:  # Display the first 5 examples
    print(entry)


('Sirius is one of the brightest stars.', {'entities': [(0, 6, 'CELESTIAL_OBJECT')]})
('Point the telescope at Canopus.', {'entities': [(23, 30, 'CELESTIAL_OBJECT')]})
('The star Rigil Kentaurus is quite famous.', {'entities': [(9, 24, 'CELESTIAL_OBJECT')]})
('Astronomers often observe Arcturus.', {'entities': [(26, 34, 'CELESTIAL_OBJECT')]})
('The star Vega is quite famous.', {'entities': [(9, 13, 'CELESTIAL_OBJECT')]})


In [2]:
import pickle

# Save the training data to a pickle file
with open("train_data.pkl", "wb") as f:
    pickle.dump(train_data, f)


In [7]:
import spacy
from spacy.lookups import Lookups
from spacy import util

# Load the language model
nlp = spacy.load("en_core_web_sm")

# Load lookups
lookups = Lookups()


# Add the lookups to the pipeline
nlp.vocab.lookups = lookups

# Now proceed with training


In [10]:
import spacy
import random
import pickle
from spacy.training import Example

# Load your training data from the pickle file
with open("train_data.pkl", "rb") as f:
    TRAIN_DATA = pickle.load(f)

# Load the SpaCy pre-trained model
nlp = spacy.load("en_core_web_sm")  # You can use other base models as well

# Get the Named Entity Recognition (NER) pipeline component
ner = nlp.get_pipe("ner")

# Add the new label (if not already present in the model)
ner.add_label("CELESTIAL_OBJECT")

# Disable other pipeline components to only train NER
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

# Training loop
with nlp.disable_pipes(*unaffected_pipes):  # Disable other components
    optimizer = nlp.begin_training()
    for itn in range(20):  # Training for 20 iterations
        print(f"Starting iteration {itn}")
        random.shuffle(TRAIN_DATA)  # Shuffle training data for randomness
        losses = {}
        for text, annotations in TRAIN_DATA:
            # Create a doc object from the input text
            doc = nlp.make_doc(text)

            # Create an example from the doc and annotations
            example = Example.from_dict(doc, annotations)

            # Update the model with the example
            nlp.update([example], drop=0.35, losses=losses)
        
        print(f"Losses after iteration {itn}: {losses}")

# Save the trained model to disk
nlp.to_disk("trained_ner_model")
print("Model saved to 'trained_ner_model'")


ValueError: [E955] Can't find table(s) lexeme_norm for language 'en' in spacy-lookups-data. Make sure you have the package installed or provide your own lookup tables if no default lookups are available for your language.